In [1]:
%load_ext rpy2.ipython

In [ ]:
%%R
install.packages("sf")
#install.packages("mapview")

(as ‘lib’ is unspecified)








































In [4]:
# prompt: open google drive folders

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
%%R
library(tidyverse)
library(sf)
#library(mapview)


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Importantando h3 

obs: para facilidar posteriormente os hexagonos já estão com os dados agregados do cnefe

In [7]:
%%R
hex_sp <- sf::read_sf("/content/drive/MyDrive/Mestrado/PTR5927 - Análise Espacial (2024)/trabalho/data/shapeFiles/sad69_hex_sp_cnefe_agg") %>% st_transform(31983)


importando camada de quadras

In [9]:
%%R

quadras <- sf::read_sf("/content/drive/MyDrive/Mestrado/PTR5927 - Análise Espacial (2024)/trabalho/data/shapeFiles/SIRGAS_SHP_quadraMDSF") %>% st_set_crs(31983)

cruzando o centroide das quadras com a base hex,

In [13]:
%%R

hex_centroides_quadras <-
quadras %>%
  st_centroid() %>%
  st_intersection(hex_sp) %>%
  st_drop_geometry()

Em seguida, a base de quadras foi importada. As geometrias foram transformadas para os respectivos centroides; e então agregadas pelo hexágono de intersecção.

In [10]:
%%R

iptu <- read.csv("/content/drive/MyDrive/Mestrado/PTR5927 - Análise Espacial (2024)/trabalho/data/IPTU_2024.csv", sep = ";")

filtrando a base de IPTU

A base do IPTU 2024 foi importada e filtrada de acordo com a informação disponível no campo "TIPO DE USO DO IMÓVEL". Em seguida, os lançamentos de IPTU foram georreferenciados de acordo com a quadra fiscal e, assim, foi possível atribuir um id da base de hexágonos.

In [11]:
%%R
iptu_fil <-
iptu %>%
  filter(TIPO.DE.USO.DO.IMOVEL %in% c('Apartamento em condom\xednio',
                                      'Flat residencial em condom\xednio',
                                      'N\xe3o residencial',
                                      'Pr\xe9dio de apartamento, n\xe3o em condom\xednio, de uso exclusivamente residencial',
                                      'Pr\xe9dio de apartamento, n\xe3o em condom\xednio, de uso misto (apartamentos e escrit\xf3rios e/ou consult\xf3rios), com ou sem loja (predomin\xe2ncia residencial)',
                                      'Resid\xeancia',
                                      'Resid\xeancia e outro uso (predomin\xe2ncia residencial)')) %>%
  mutate(qd_setor = substr(NUMERO.DO.CONTRIBUINTE,1,3), qd_fiscal = substr(NUMERO.DO.CONTRIBUINTE,4,6), cd_lote = substr(NUMERO.DO.CONTRIBUINTE,7,10)) %>%
  select(qd_setor, qd_fiscal, cd_lote)

In [ ]:
Em seguida, os dados de lançamentos de IPTU foram trazidos à base de hexágonos - que, lembre-se, já conta com os dados do CNEFE. Assim, foi possível comparar o número de endereços residenciais pelo CNEFE; e o número de lançamentos de IPTU com características de uso residencial. O gráfico a seguir resume os resultados.


In [14]:
%%R
hex_lancamentos_iptu <-
iptu_fil %>%
  left_join(hex_centroides_quadras) %>%
  group_by(fid) %>%
  summarise(n_lancamentos = n())

Joining with `by = join_by(qd_setor, qd_fiscal)`


In [16]:
%%R
hex_iptu <- hex_lancamentos_iptu %>%
  right_join(hex_sp) %>%
  st_as_sf() %>%
  st_make_valid()

Joining with `by = join_by(fid)`


In [19]:
%%R
hex_iptu %>%  st_write("/content/drive/MyDrive/Mestrado/PTR5927 - Análise Espacial (2024)/trabalho/data/shapeFiles/SIRGA_hex_iptu/SIRGA_hex_iptu.shp")

Layer SIRGA_hex_iptu in dataset /content/drive/MyDrive/Mestrado/PTR5927 - Análise Espacial (2024)/trabalho/data/shapeFiles/SIRGA_hex_iptu/SIRGA_hex_iptu.shp already exists:
use either append=TRUE to append to layer or append=FALSE to overwrite layer


  Dataset already exists.


 




Error in (function (expr, envir = parent.frame(), enclos = if (is.list(envir) ||  : 
  Dataset already exists.


##todo
- Filtrar apenas colunas utilizadas (esp_1 and qnt de lançamentos) 

In [ ]:
%%R
# Write the GeoDataFrame to a shapefile
hex_iptu.to_file("/content/drive/MyDrive/Mestrado/PTR5927 - Análise Espacial (2024)/trabalho/data/shapeFiles/SIRGA_hex_iptu/SIRGA_hex_iptu.shp", driver='ESRI Shapefile')
